# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [15]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
Cities_Data = pd.read_csv("output_data/Cities_Data.csv", encoding = "utf-8")
Cities_Data.head()

,city_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind speed,Country,Date
0,0,port lincoln,-34.7333,135.8667,54.50,67,78,5.01,AU,1657358160
1,1,castro,-24.7911,-50.0119,53.76,95,86,7.09,BR,1657358161
2,2,loikaw,19.6742,97.2094,79.23,78,100,3.62,MM,1657358161
3,3,codrington,-38.2667,141.9667,47.61,83,6,3.51,AU,1657358161
4,4,nossa senhora da gloria,-10.2183,-37.4203,63.70,95,38,4.45,BR,1657358162


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
Humidity = Cities_Data["Humidity"].astype(float)
Max_Humidity = Humidity.max()
Max_Humidity

100.0

In [4]:
locations = Cities_Data[["Lat","Lng"]].astype(float)
locations.head()

,Lat,Lng
0,-34.7333,135.8667
1,-24.7911,-50.0119
2,19.6742,97.2094
3,-38.2667,141.9667
4,-10.2183,-37.4203


In [6]:
#plotting heatmap
fig = gmaps.figure( center = [0,0], zoom_level = 2)
#creating and adding heat layer
heat_layer = gmaps.heatmap_layer(locations,weights=Humidity, dissipating= False, max_intensity = Max_Humidity, point_radius = 3)
fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
narrow_df = Cities_Data.loc[(Cities_Data["Max Temp"]>65) & (Cities_Data["Max Temp"]<80) & (Cities_Data["Cloudiness"]== 0) & (Cities_Data["Wind speed"]<=6),:]
narrow_df = narrow_df.dropna(how="any")
narrow_df.head()

,city_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind speed,Country,Date
7,7,east london,-33.0153,27.9116,76.14,29,0,2.30,ZA,1657358100
67,67,jasper,37.2001,-94.3502,75.25,86,0,3.44,US,1657358147
125,125,elk grove,38.4088,-121.3716,68.11,73,0,5.75,US,1657358191
128,128,horasan,40.0458,42.1728,78.66,40,0,3.06,TR,1657358192
139,139,saint-francois,46.4154,3.9054,71.67,56,0,4.61,FR,1657358196


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [21]:
#hotel_df DataFrame
hotel_df = narrow_df.reset_index(drop=True)
hotel_df["Hotel Name"]= ""
hotel_df.head()

,city_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind speed,Country,Date,Hotel Name
0,7,east london,-33.0153,27.9116,76.14,29,0,2.30,ZA,1657358100,
1,67,jasper,37.2001,-94.3502,75.25,86,0,3.44,US,1657358147,
2,125,elk grove,38.4088,-121.3716,68.11,73,0,5.75,US,1657358191,
3,128,horasan,40.0458,42.1728,78.66,40,0,3.06,TR,1657358192,
4,139,saint-francois,46.4154,3.9054,71.67,56,0,4.61,FR,1657358196,


In [33]:
#target url
target_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
#setting parameters dic  
params = {
    "radius" : 5000,
    "keywords":"hotels",
    "types" : "hotel",
    "key" : g_key }

#identifying hotel using lat and lng
for index, row in hotel_df.iterrows():
    
    #get the lat and lng in each row
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    #changing the location at each iteration with fix params
    params["location"] = f"{latitude},{longitude}"
    
    
    #converting response into a json file
    hotel_name = requests.get(target_url,params=params).json()
    #print(json.dumps(hotel_name,indent=4, sort_keys=True))
    
    try: 
        hotel_df.loc[index, "Hotel Name"]= hotel_name["results"][0]["name"]
    except(KeyError, IndexError):
        print("Missing field/result... skipping.")
hotel_df.head()       
        

,city_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind speed,Country,Date,Hotel Name
0,7,east london,-33.0153,27.9116,76.14,29,0,2.30,ZA,1657358100,East London
1,67,jasper,37.2001,-94.3502,75.25,86,0,3.44,US,1657358147,Carthage
2,125,elk grove,38.4088,-121.3716,68.11,73,0,5.75,US,1657358191,Elk Grove
3,128,horasan,40.0458,42.1728,78.66,40,0,3.06,TR,1657358192,Horasan
4,139,saint-francois,46.4154,3.9054,71.67,56,0,4.61,FR,1657358196,Molinet


In [30]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [32]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,
    info_box_content=[f"Hotel location: {rate}" for rate in hotel_info])

fig.add_layer(markers)
# Display figure
fig


Figure(layout=FigureLayout(height='420px'))